In [ ]:
import json
from functools import partial
from typing import List

import matplotlib.pyplot as plt

import collections

import numpy as np
import pandas as pd
import seaborn as sns

#from catboost import CatBoostClassifier, Pool
#from catboost.utils import eval_metric

#from transformers import BertModel, BertTokenizer
#import torch

from scipy.spatial.distance import cosine, euclidean

from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity


from sklearn.model_selection import train_test_split

### Load data

In [ ]:
pairs = pd.read_parquet("hackathon_files_for_participants_ozon/train_pairs.parquet")
products_all = pd.read_parquet("hackathon_files_for_participants_ozon/train_data.parquet")

In [ ]:
products_all.categories = products_all.categories.apply(lambda x: json.loads(x)["3"])

In [ ]:
pairs["target"].value_counts()

In [ ]:
c = products_all[['categories', 'variantid']].groupby(
    'categories'
).count(
).sort_values(
    by="variantid",
    ascending=0
)

fig, (ax) = plt.subplots(1, 1, figsize=(10, 40))

ax.barh(c.index, c.variantid.values)

In [ ]:
products = products_all[products_all.categories == "Смартфоны, планшеты, мобильные телефоны"]

Get raw data for each variantid.

In [ ]:
product_pairs = pairs.merge(
    products.add_suffix('1'),
    on="variantid1"
).merge(
    products.add_suffix('2'),
    on="variantid2"
).dropna(subset=[
    'name1', 'name2',
    'characteristic_attributes_mapping1',
    'characteristic_attributes_mapping2'
])

In [ ]:
identcial_pairs = product_pairs[product_pairs['target'] == 1]
various_pairs = product_pairs[product_pairs['target'] == 0]

identcial_pairs.shape[0], various_pairs.shape[0]

In [ ]:
feats_set3 = [
    'variantid1', 
    'name1', 
    'characteristic_attributes_mapping1',
    'variantid2',
    'name2', 
    'characteristic_attributes_mapping2',
    'target'
]

In [ ]:

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

sentence1 = "Зарядное устройство\nКабель USB - Lightning"
sentence2 = "Зарядное устройство\nКабель USB - Type-C"

encoded_sentence1 = tokenizer.encode_plus(sentence1, add_special_tokens=True, return_tensors='pt')
encoded_sentence2 = tokenizer.encode_plus(sentence2, add_special_tokens=True, return_tensors='pt')

with torch.no_grad():
    model_output1 = model(encoded_sentence1['input_ids'], attention_mask=encoded_sentence1['attention_mask'])
    model_output2 = model(encoded_sentence2['input_ids'], attention_mask=encoded_sentence2['attention_mask'])
embeddings1 = model_output1[0][:, 0, :]
embeddings2 = model_output2[0][:, 0, :]

print(cosine_similarity(embeddings1, embeddings2))

In [ ]:
a = [
    (
        sum([
            len(set(
                    json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping1
                    )[key]
                ) & set(
                    json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping2
                    )[key]
                )
            ) for key in list(
                set(
                    json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping1
                    )
                ) & set(
                    json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping2
                    )
                )
            )
        ]) / max(
            sum(len(v) for v in json.loads(product_pairs.loc[i].characteristic_attributes_mapping1).values()), 
            sum(len(v) for v in json.loads(product_pairs.loc[i].characteristic_attributes_mapping2).values())
        )
    ) for i in np.random.choice(various_pairs.index, 5000)
]

b = [
    (
        sum([
            len(set(
                    json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping1
                    )[key]
                ) & set(
                    json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping2
                    )[key]
                )
            ) for key in list(
                set(
                    json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping1
                    )
                ) & set(
                    json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping2
                    )
                )
            )
        ]) / max(
            sum(len(v) for v in json.loads(product_pairs.loc[i].characteristic_attributes_mapping1).values()), 
            sum(len(v) for v in json.loads(product_pairs.loc[i].characteristic_attributes_mapping2).values())
        )
    ) for i in np.random.choice(identcial_pairs.index, 5000)
]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.set_ylim(0,700)
ax2.set_ylim(0,700)

ax1.hist(a, bins=100)
ax1.set_title('совпадение аттрибутов различных товаров\n из категории "смартфоны"')

ax2.hist(b, bins=100)
ax2.set_title('совпадение аттрибутов одинаковых товаров\n из категории "смартфоны"')

In [ ]:
various_pairs.loc[rnd]

In [ ]:
rnd = np.random.choice(various_pairs.index)
a1 = json.loads(
    various_pairs.loc[rnd].characteristic_attributes_mapping1
)
a2 = json.loads(
    various_pairs.loc[rnd].characteristic_attributes_mapping2
)
(various_pairs.loc[rnd].name1, various_pairs.loc[rnd].name2),

# a1, a2, {
#     key: set(a1[key]) & set(a2[key])
#     for key in list(set(a1) & set(a2))
# }

In [ ]:
from itertools import chain

In [ ]:
various_pairs

In [ ]:
a = [i for i in zip([1,2,3], [2,34,5], [2,1,3])]
a


In [ ]:
importance_attr = collections.Counter(
    attr_name if json.loads(attributes1)[attr_name] != json.loads(attributes2)[attr_name] else 'matched'
    for (attributes1, attributes2) in zip(various_pairs.characteristic_attributes_mapping1, 
                                          various_pairs.characteristic_attributes_mapping2)
        for attr_name in set(json.loads(attributes1)) & set(json.loads(attributes2))
)

In [ ]:
fig, (ax) = plt.subplots(1, 1, figsize=(20, 40))

ax.barh(
    list(dict(importance_attr.most_common()).keys())[1:], 
    list(dict(importance_attr.most_common()).values())[1:]
)
fig.savefig('./tmp.png')

In [ ]:
share_counter = collections.Counter(
    attr_name 
    for attributes in products.characteristic_attributes_mapping
        for attr_name in json.loads(attributes).keys()
)

fig, (ax) = plt.subplots(1, 1, figsize=(10, 40))

ax.barh(
    list(dict(share_counter.most_common()).keys()), 
    dict(share_counter.most_common()).values()
)

In [ ]:
rnd = np.random.choice(product_pairs.index)

name1=product_pairs.loc[rnd].name1
name2=product_pairs.loc[rnd].name2

words_in_name = name.translate(str.maketrans('', '', '_()')).replace('/',' ').lower().split(' ')

s = ""
s += f"target: {product_pairs.loc[rnd].target}\n"
s += f"name1:  {name1}\n"
s += f"name2:  {name2}\n"
name1_attr2 = set([
    word 
    for word in product_pairs.loc[rnd].name1.translate(
        str.maketrans('', '', '_()')
        ).replace('/',' ').lower().split(' ') if any(
            word in sentence 
            for sentence in [
                ' '.join(l)
                for l in json.loads(
                    product_pairs.loc[rnd].characteristic_attributes_mapping1.lower()
                ).values()
            ]
        )
    ]
)
s += f"name1 & attr2: {name1_attr2}\n"

name1_attr1 = set([
        word 
        for word in product_pairs.loc[rnd].name1.translate(str.maketrans('', '', '_()')
                                                          ).replace('/',' '
                                                          ).lower(
                                                          ).split(' '
                                                          ) if any(
            word in sentence 
            for sentence in [
                ' '.join(l)
                for l in json.loads(
                    product_pairs.loc[rnd].characteristic_attributes_mapping2.lower()
                ).values()
            ]
        )
    ]
)

s += f"name1 & attr1 : {name1_attr1}\n"

attr1 = json.dumps(json.loads(
    product_pairs.loc[rnd].characteristic_attributes_mapping1.lower()
))

attr2 = json.dumps(json.loads(
    product_pairs.loc[rnd].characteristic_attributes_mapping2.lower()
))

#s += f"attr1: {attr1}\n"
#s += f"attr2: {attr2}\n"

print(s)

In [ ]:
rnd = np.random.choice(product_pairs.index)

name=product_pairs.loc[rnd].name1

words_in_name = name.translate(str.maketrans('', '', '_()')).replace('/',' ').lower().split(' ')

1=[
    (
        len(set([
            word 
            for word in product_pairs.loc[i].name1.translate(str.maketrans('', '', '_()')
                                                            ).replace('/',' '
                                                            ).lower().split(' ') if any(
                word in sentence 
                for sentence in [
                    ' '.join(l)
                    for l in json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping2.lower()
                    ).values()
                ]
            )
        ]) & set(words_in_name))/len(words_in_name)
    ) for i in np.random.choice(identcial_pairs.index, 3000)
]


b=[
    (
        len(set([
            word 
            for word in product_pairs.loc[i].name1.translate(str.maketrans('', '', '_()')
                                                            ).replace('/',' '
                                                            ).lower().split(' ') if any(
                word in sentence 
                for sentence in [
                    ' '.join(l)
                    for l in json.loads(
                        product_pairs.loc[i].characteristic_attributes_mapping1.lower()
                    ).values()
                ]
            )
        ]) & set(words_in_name))/len(words_in_name)
    ) for i in np.random.choice(various_pairs.index, 3000)
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
#ax1.set_ylim(0,700)
#ax2.set_ylim(0,700)

ax1.hist(a, bins=10)
#ax1.set_title('совпадение аттрибутов различных товаров\n из категории "смартфоны"')

ax2.hist(b, bins=10)
#ax2.set_title('совпадение аттрибутов одинаковых товаров\n из категории "смартфоны"')
plt.show()

In [ ]:
set([
    word 
    for word in words_in_name if any(
        word in sentence 
        for sentence in [
            ' '.join(l)
            for l in json.loads(
                product_pairs.loc[rnd].characteristic_attributes_mapping1.lower()
            ).values()
        ]
    )
]) & set(words_in_name)

In [ ]:
rnd = np.random.choice(product_pairs.index)

print(product_pairs.loc[rnd].target)
{
    product_pairs.loc[rnd].name1 : (
        json.loads(product_pairs.loc[rnd].characteristic_attributes_mapping1)
    ),
    product_pairs.loc[rnd].name2 : (
        json.loads(product_pairs.loc[rnd].characteristic_attributes_mapping2)
    )
}

In [ ]:
sum(
    [
        len(set(a1[key]) & set(a2[key])) 
        for key in list(set(a1) & set(a2))
    ]
) / max(
    sum(len(v) for v in a1.values()), 
    sum(len(v) for v in a2.values())
)

In [ ]:
product_pairs.name1.shape[0] - product_pairs.name1.unique().shape[0]

In [ ]:
ncf = various_pairs[
    ['characteristic_attributes_mapping1','characteristic_attributes_mapping2']
]
ncf = ncf.dropna()
ncf.shape

In [ ]:
len(cat_not_match), len(ncat_not_match)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
(various_pairs.categories1 != various_pairs.categories2).values.nonzero()[0].shape

In [ ]:
(identcial_pairs.categories1 != identcial_pairs.categories2).values.nonzero()[0].shape

In [ ]:
collusions = identcial_pairs[identcial_pairs.categories1 != identcial_pairs.categories2]

In [ ]:
col_bert = various_pairs[['name_bert_641', 'name_bert_642']]

In [ ]:
col_bert

In [ ]:
identcial_pairs['similarity'] = [ 
    cosine_similarity(
        identcial_pairs.loc[i].name_bert_641.reshape(1,-1),
        identcial_pairs.loc[i].name_bert_642.reshape(1,-1)
    )[0][0]
    for i in identcial_pairs.index
]

In [ ]:
various_pairs['similarity'] = [ 
    cosine_similarity(
        various_pairs.loc[i].name_bert_641.reshape(1,-1),
        various_pairs.loc[i].name_bert_642.reshape(1,-1)
    )[0][0] 
    for i in various_pairs.index
]

In [ ]:

fig, ax = plt.subplots()

ax.set_xlim(0.98, 1)
ax.set_ylim(0, 6000)

ax.grid(True, color='grey', linestyle='--', linewidth=0.5)
ax.hist(identcial_pairs.similarity, bins=1000, color='r')

ax.grid(True, color='grey', linestyle='--', linewidth=0.5)
ax.hist(various_pairs.similarity, bins=1000, color="black", alpha=0.4)

plt.show()

In [ ]:
plt.scatter(identcial_pairs.index, 
            identcial_pairs['similarity'],
            linewidths=0.5
            )

In [ ]:
plt.scatter(various_pairs.index,
            various_pairs['similarity'],
            linewidths=0.5
            )

In [ ]:
json.loadsimilarityusions.loc[59].characteristic_attributes_mapping1)

In [ ]:
json.loads(collusions.loc[59].characteristic_attributes_mapping2)

In [ ]:
collusions.loc[59].name2

In [ ]:
various_pairs[various_pairs.categories1 == various_pairs.cat egories2]

In [ ]:
various_pairs[various_pairs.categories1 == various_pairs.categories2]

In [ ]:
(product_pairs.loc[60].variantid1, product_pairs.loc[60].variantid2, product_pairs.loc[60].target),\
(product_pairs.loc[61].variantid1, product_pairs.loc[61].variantid2, product_pairs.loc[61].target)

In [ ]:
(product_pairs.loc[60].name1, product_pairs.loc[60].name2),\
(product_pairs.loc[61].name1, product_pairs.loc[61].name2)

In [ ]:
(product_pairs.loc[60].categories1, product_pairs.loc[60].categories2),\
(product_pairs.loc[61].categories1, product_pairs.loc[61].categories2)

In [ ]:
similarity60 = cosine_similarity(
    product_pairs.loc[60].main_pic_embeddings_resnet_v11[0].reshape(1,-1),
    product_pairs.loc[60].main_pic_embeddings_resnet_v12[0].reshape(1,-1)
)[0][0]
similarity61 = cosine_similarity(
    product_pairs.loc[61].main_pic_embeddings_resnet_v11[0].reshape(1,-1),
    product_pairs.loc[61].main_pic_embeddings_resnet_v12[0].reshape(1,-1)
)[0][0]
similarity60, similarity61

In [ ]:
json.loads(product_pairs.loc[60].characteristic_attributes_mapping1),\
json.loads(product_pairs.loc[60].characteristic_attributes_mapping2),\
json.loads(product_pairs.loc[61].characteristic_attributes_mapping1),\
json.loads(product_pairs.loc[61].characteristic_attributes_mapping2)

In [ ]:
features.categories1.loc[60],\
features.categories2.loc[60]

In [ ]:
features.categories1.loc[61],\
features.categories2.loc[61]

In [ ]:
json.loads(ncf.characteristic_attributes_mapping1[0]),\
json.loads(ncf.characteristic_attributes_mapping2[0])

In [ ]:
set(
    map(
        tuple,
        json.loads(
            ncf.characteristic_attributes_mapping1[0]
        ).values()
    )
) & set(
    map(
        tuple,
        json.loads(
            ncf.characteristic_attributes_mapping2[0]
        ).values()
    )
)

In [ ]:
set(map(tuple, json.loads(
    cf.characteristic_attributes_mapping1[2]
).values())) 

In [ ]:
match_char = [
    (
        len(set(map(tuple,
                    json.loads(
                        cf.characteristic_attributes_mapping1[i]
                    ).values()
                )
            ) & set(map(tuple,
                    json.loads(
                        cf.characteristic_attributes_mapping2[i]
                    ).values()
                )
            )
        ),
        min(
            len(json.loads(cf.characteristic_attributes_mapping1[i]).keys()),
            len(json.loads(cf.characteristic_attributes_mapping2[i]).keys())
        )
    ) for i in cf.index
] 

In [ ]:
match_char[:10]

In [ ]:
np.mean([(x-y)**2 for x,y in match_char])

In [ ]:
neg_match_char = [
    (
        len(set(map(tuple,
                    json.loads(
                        ncf.characteristic_attributes_mapping1[i]
                    ).values()
                )
            ) & set(map(tuple,
                    json.loads(
                        ncf.characteristic_attributes_mapping2[i]
                    ).values()
                )
            )
        ),
        min(
            len(json.loads(ncf.characteristic_attributes_mapping1[i]).keys()),
            len(json.loads(ncf.characteristic_attributes_mapping2[i]).keys())
        )
    ) for i in ncf.index
] 

In [ ]:
np.mean([(x-y)**2 for x,y in neg_match_char])

In [ ]:
neg_match_char[:10]

In [ ]:
features.head(2)

Features functions.

In [ ]:
def get_pic_features(main_pic_embeddings_1,
                     main_pic_embeddings_2,
                     percentiles: List[int]):
    """Calculate distances percentiles for 
    pairwise pic distances. Percentiles are useful 
    when product has several pictures.
    """
    
    if main_pic_embeddings_1 is not None and main_pic_embeddings_2 is not None:
        main_pic_embeddings_1 = np.array([x for x in main_pic_embeddings_1])
        main_pic_embeddings_2 = np.array([x for x in main_pic_embeddings_2])
        
        dist_m = pairwise_distances(
            main_pic_embeddings_1, main_pic_embeddings_2
        )
    else:
        dist_m = np.array([[-1]])

    pair_features = []
    pair_features += np.percentile(dist_m, percentiles).tolist()

    return pair_features


def text_dense_distances(ozon_embedding, comp_embedding):
    """Calculate Euclidean and Cosine distances between
    ozon_embedding and comp_embedding.
    """
    pair_features = []
    if ozon_embedding is None or comp_embedding is None:
        pair_features = [-1, -1]
    elif len(ozon_embedding) == 0 or len(comp_embedding) == 0:
        pair_features = [-1, -1]
    else:
        pair_features.append(
            euclidean(ozon_embedding, comp_embedding)
        )
        cosine_value = cosine(ozon_embedding, comp_embedding)
        
        pair_features.append(cosine_value)

    return pair_features


In [ ]:
get_pic_features_func = partial(
    get_pic_features,
    percentiles=[0, 25, 50]
)

In [ ]:
features[["pic_dist_0_perc", "pic_dist_25_perc", "pic_dist_50_perc"]] = (
    features[["pic_embeddings_resnet_v11", "pic_embeddings_resnet_v12"]].apply(
        lambda x: pd.Series(get_pic_features_func(*x)), axis=1
    )
)

# btw try to add distances between main pic embs

In [ ]:
features[["euclidean_name_bert_dist", "cosine_name_bert_dist"]] = (
    features[["name_bert_641", "name_bert_642"]].apply(
        lambda x: pd.Series(text_dense_distances(*x)), axis=1
    )
)

# try to use your favorite NLP model

In [ ]:
features["cat3"] = features["categories1"].apply(lambda x: json.loads(x)["3"])
cat3_counts = features["cat3"].value_counts().to_dict()

Let's find good cat size threshold

In [ ]:
cntr = 0
for cat3 in cat3_counts:
    if cat3_counts[cat3] < 1_000:
        cntr += cat3_counts[cat3]
        
cntr

10k for "rest" cats probably is good

In [ ]:
features["cat3_grouped"] = features["cat3"].apply(lambda x: x if cat3_counts[x] > 1000 else "rest")

Choose features

In [ ]:
feats = ["pic_dist_0_perc", "pic_dist_25_perc", "pic_dist_50_perc", 
         "euclidean_name_bert_dist", "cosine_name_bert_dist"]

Split dataset

In [ ]:
X_train, X_test = train_test_split(
    features[feats + ["target", "variantid1", "variantid2", "cat3_grouped"]], 
    test_size=0.1, random_state=42, stratify=features[["target", "cat3_grouped"]]
)

X_train, X_val = train_test_split(
    X_train[feats + ["target",  "variantid1", "variantid2", "cat3_grouped"]], 
    test_size=0.1, random_state=42, stratify=X_train[["target", "cat3_grouped"]]
)

y_test = X_test[["target", "variantid1", "variantid2"]]
X_test = X_test.drop(["target"], axis=1)

y_train = X_train["target"]
y_val = X_val["target"]

X_train = X_train.drop(["target"], axis=1)
X_val = X_val.drop(["target"], axis=1)


train_pool = Pool(
    data=X_train[feats],
    label=y_train,
)
eval_pool = Pool(
    data=X_val[feats],
    label=y_val,
)

Train model

In [ ]:
model = CatBoostClassifier()

model.fit(
    train_pool,
    eval_set=eval_pool,
    plot=True,
    verbose=True,
    use_best_model=True,
    early_stopping_rounds=50,
    metric_period=100
)
model.save_model(f"./models/baseline.cbm")

### Metric

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc


def pr_auc_macro(
    target_df: pd.DataFrame,
    predictions_df: pd.DataFrame,
    prec_level: float = 0.75,
    cat_column: str = "cat3_grouped"
) -> float:
    
    df = target_df.merge(predictions_df, on=["variantid1", "variantid2"])
    
    y_true = df["target"]
    y_pred = df["scores"]
    categories = df[cat_column]
    
    weights = []
    pr_aucs = []

    unique_cats, counts = np.unique(categories, return_counts=True)

    for i, category in enumerate(unique_cats):
        cat_idx = np.where(categories == category)[0]
        y_pred_cat = y_pred[cat_idx]
        y_true_cat = y_true[cat_idx]

        y, x, thr = precision_recall_curve(y_true_cat, y_pred_cat)
        gt_prec_level_idx = np.where(y >= prec_level)[0]

        try:
            pr_auc_prec_level = auc(x[gt_prec_level_idx], y[gt_prec_level_idx])
            if not np.isnan(pr_auc_prec_level):
                pr_aucs.append(pr_auc_prec_level)
                weights.append(counts[i] / len(categories))
        except ValueError as err:
            pr_aucs.append(0)
            weights.append(0)
    return np.average(pr_aucs, weights=weights)

In [ ]:
X_test["scores"] = model.predict_proba(X_test[feats])[:, 1]

In [ ]:
pr_auc_macro_metr = pr_auc_macro(
    target_df=y_test, 
    predictions_df=X_test,
    prec_level=0.75,
    cat_column="cat3_grouped"
)

pr_auc_macro_metr

In [ ]:
precision, recall, thrs = precision_recall_curve(y_test["target"], X_test["scores"])
pr_auc = auc(recall, precision)

fig, ax1 = plt.subplots(1, figsize=(15, 7))

ax1.plot(recall, precision)
ax1.axhline(y=0.75, color='grey', linestyle='-');

### Submission

In [ ]:
test_pairs = pd.read_parquet("hackathon_files_for_participants_ozon/test_pairs_wo_target.parquet")
test_etl = pd.read_parquet("hackathon_files_for_participants_ozon/test_data.parquet")

Generate the same features as for train

In [ ]:
test_features = (
    test_pairs
    .merge(
        test_etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        test_etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [ ]:
test_features[["pic_dist_0_perc", "pic_dist_25_perc", "pic_dist_50_perc"]] = (
    test_features[["main_pic_embeddings_resnet_v11", "main_pic_embeddings_resnet_v12"]].apply(
        lambda x: pd.Series(get_pic_features_func(*x)), axis=1
    )
)

test_features[["euclidean_name_bert_dist", "cosine_name_bert_dist"]] = (
    test_features[["name_bert_641", "name_bert_642"]].apply(
        lambda x: pd.Series(text_dense_distances(*x)), axis=1
    )
)

In [ ]:
test_features["cat3"] = test_features["categories1"].apply(lambda x: json.loads(x)["3"])

In [ ]:
test_features.shape

In [ ]:
test_cat3_counts = test_features["cat3"].value_counts().to_dict()

cntr = 0
for cat3 in test_cat3_counts:
    if test_cat3_counts[cat3] < 50:
        cntr += test_cat3_counts[cat3]
        
cntr

In [ ]:
test_features["cat3_grouped"] = test_features["cat3"].apply(lambda x: x if test_cat3_counts[x] > 50 else "rest")
# btw you can rename to `rest` the same categories that were assigned to this group in train

In [ ]:
submission_example = test_features.copy()

In [ ]:
submission_example["scores"] = model.predict_proba(test_features[feats])[:, 1]

In [ ]:
submission_example = submission_example[["variantid1", "variantid2", "scores"]]

In [ ]:
submission_example.drop_duplicates().merge(
    test_features[["variantid1", "variantid2"]].drop_duplicates(["variantid1", "variantid2"]),
    on=["variantid1", "variantid2"]
).to_csv("submission_example.csv", index=False)

Upload your submission to leaderboard :)

### Recommendations 

- Work with names, in electronics names often contain a lot of useful information for matching.
- Don't forget about attributes: working with it will allow your model to better distinguish matches from non-matches.